In [ ]:
import pandas as pd
import warnings
# Filter out all warning messages
warnings.filterwarnings("ignore")

df=pd.DataFrame()
# Use pd.concat instead of append
df = pd.concat([df, pd.DataFrame([{"1": 1, "2": 2, "3": 3}])], ignore_index=True)
df

In [ ]:
import os
import pandas as pd

# File path
file_path = r"D:\RERA op\rera main code\o1b_20241231_rera_final_review_Pune.csv"

# Check if the file exists
if not os.path.exists(file_path):
    # Create a blank DataFrame with the "RERA ID" column
    blank_df = pd.DataFrame(columns=["RERA ID"])
    blank_df.to_csv(file_path, index=False)
    print(f"File did not exist. A blank file with 'RERA ID' column has been created at: {file_path}")

# Load the file"
df_rera_unique = pd.read_csv(file_path)
print(df_rera_unique)

In [ ]:
# pip install pandas==1.5.3

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import requests
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from time import sleep
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException, TimeoutException
import time
from PIL import Image, ImageEnhance, ImageFilter
import pytesseract
import re

# Scrapping

In [ ]:
# def project_detail_page(url):
#     webpage = requests.get(href,10)
#     fsoup = BeautifulSoup(webpage.content, "html.parser")
#     return fsoup

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def project_detail_page(url):
    # Set up the web driver
    option = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options = option)

    # Open the web page
    driver.get(url)

    # Wait for the page to fully load (you can modify the condition as needed)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

    # Get the page source
    page_source = driver.page_source
    # Parse the page source with BeautifulSoup
    fsoup = BeautifulSoup(page_source, "html.parser")
    return fsoup


In [ ]:
import re
import PyPDF2

def extract_dates_from_pdf(file_path):
    """
    Extracts specific date patterns from a PDF file.

    Parameters:
        file_path (str): The path to the PDF file.

    Returns:
        list: A list of extracted dates in the format DD/MM/YYYY.
    """
    date = []  # List to store extracted dates

    try:
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text = page.extract_text()
                if text:  # Ensure the page has text
                    # Normalize the text by replacing multiple spaces with a single space
                    normalized_text = re.sub(r'\s+', ' ', text)

                    # Find dates with specific patterns
                    dat = re.findall(r"Dated: (\d{2}/\d{2}/\d{4})", normalized_text)
                    date.extend(dat)
                    
                    cfrom = re.findall(r"commencing from (\d{2}/\d{2}/\d{4})", normalized_text)
                    date.extend(cfrom)
                    
                    end = re.findall(r"ending with (\d{2}/\d{2}/\d{4})", normalized_text)
                    date.extend(end)

        return date if date else None  # Return dates or None if no matches found
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
def rera_data(fsoup):
    df_past = pd.DataFrame(columns = ["Organization","Owners/Patrners","Contact","Project","Project Name","Type of Project",
                                    "Others","Land Area (in sqmtrs)","Address","CTS number","Number of Buildings/Plot",
                                    "Number Apartments","Original Proposed Date of Completion","Actual Date of Completion",
                                    "website of Organization"])
    # go to where project details table is
    content = fsoup.find('div',id = "DivProject").find_all('div',class_="x_content label-block")

    # details Litigations related to the project
    part1 = content[0].find_all('div',class_="row")

    # FSI Details "Sanctioned FSI of the project applied for registration"
    part2 = content[2].find_all('div',class_='row')


    # Bank Details "Bank Name"
    part3 = content[3].find_all('div',class_="row")


    # ******project details
    temp = part1[0].find_all('div',class_="col-md-3 col-sm-3")



    # project name
    name=""
    try:
        name = temp[1].text.strip()

    except:

        name="NA"
    print(name)
    status =""
    try:
        # status of project
        status = temp[3].text.strip()

    except:
        status="NA"



    # ******dates Related to projects
    temp = part1[1].find_all('div',class_="col-md-3 col-sm-3")

    date=""
    try:
        # Proposed Date of Completion
        date = temp[1].text.strip()

    except:
        date="NA"


    rdate=""
    try:
        # Revised Proposed Date of Completion
        rdate = temp[3].text.strip()

    except:
        rdate="NA"


    # ************* Details of Litigations related to the project
    temp = part1[3].find_all('div',class_="col-md-3 col-sm-3")

    # Type of the Project
    type=""
    try:
        type = temp[3].text.strip()

    except:
        type="NA"


    # Litigation Detail
    litigations=""
    try:
        litigations = temp[1].text.strip()

    except:
        litigations="NA"


    # *************** Details of Promoter (land owner or Investor)

    temp = part1[4].find_all('div',class_="col-md-3 col-sm-3")

#     Promoter (land owner or Investor)
    promoter=""
    try:
        promoter = temp[1].text.strip()

    except:
        promoter="NA"


    # ****************
    temp = part1[6].find_all('div',class_="col-md-3 col-sm-3")

    # Plot Bearing No / CTS no  / Survey Number/Final Plot no.
    no=""
    try:
        no = temp[1].text.strip()

    except:
        no="NA"


    # Boundaries East
    try:
        east = temp[3].text.strip()

    except:
        east="NA"


    # ************************
    temp = part1[7].find_all('div',class_="col-md-3 col-sm-3")

    # Boundaries West
    try:
        west = temp[1].text.strip()

    except:
        west="NA"


    # Boundaries North
    try:
        north = temp[3].text.strip()

    except:
        north="NA"


    # *********************
    temp = part1[8].find_all('div',class_="col-md-3 col-sm-3")
    try:
        south = temp[1].text.strip()

    except:
        south="NA"
#         print("South",south)

    ####### same as North
    try:
        state = temp[3].text.strip()
#         print('State', state)
    except:
        state="NA"


    # *********************
    temp = part1[9].find_all('div',class_="col-md-3 col-sm-3")

    # division
    try:
        division = temp[1].text.strip()

    except:
        division="NA"


    # District
    try:
        district = temp[3].text.strip()

    except:
        district="NA"


    # ************************
    temp = part1[10].find_all('div',class_="col-md-3 col-sm-3")

    # taluka
    try:
        taluka = temp[1].text.strip()

    except:
        taluka="NA"


    # Village
    try:
        village = temp[3].text.strip()

    except:
        village="NA"


    # *******************
    temp = part1[11].find_all('div',class_="col-md-3 col-sm-3")

    # Street
    try:
        street = temp[1].text.strip()

    except:
        street="NA"


    # locality
    try:
        locality = temp[3].text.strip()

    except:
        locality="NA"


    # ***************
    temp = part1[12].find_all('div',class_="col-md-3 col-sm-3")

    # pincode
    try:
        pincode = temp[1].text.strip()

    except:
        pincode="NA"


    # area
    try:
        area = temp[3].text.strip()

    except:
        area="NA"

    # #Total Plot/Project area (sqmts)
    # try:
    #     label = fsoup.find('label', text='Total Plot/Project area (sqmts)')
    #     value_div = label.find_parent('div').find_next_sibling('div')

    #     # Print the extracted value
    #     print(value_div.text.strip())




    # ******************
    temp = part1[13].find_all('div',class_="col-md-3 col-sm-3")

    # Total Number of Proposed Building/Wings (In the Layout/Plot)
    try:
        total_building = temp[1].text.strip()

    except:
        total_building="NA"


    # **********************
    temp = part1[14].find_all('div',class_="col-md-3 col-sm-3")

    # Number of Sanctioned Building
    try:
        sanc_count = temp[1].text.strip()

    except:
        sanc_count="NA"


    # Proposed But Not Sanctioned Buildings Count
    try:
        notsanc_count = temp[3].text.strip()

    except:
        notsanc_count="NA"


    # ******************************
    temp = part1[15].find_all('div',class_="col-md-3 col-sm-3")

    # Total Recreational Open Space as Per Sanctioned Plan
    # Agg Area
    try:
        agg_area=temp[1].text.strip()

    except:
        agg_area="NA"





    ##-------********** Part 2

    temp = part2[0].find_all('div',class_="col-md-3 col-sm-3")

    # FSI Area
    try:
        FSI_area = temp[1].text.strip()
#         print("FSI Area",FSI_area)
    except:
        FSI_area="NA"
#         print("FSI Area",FSI_area)

    # FSI_area_approved
    try:
        FSI_area_approved = temp[3].text.strip()

    except:
        FSI_area_approved="NA"


    # ******************************
    temp = part2[1].find_all('div',class_="col-md-3 col-sm-3")

    # total_fsi
    try:
        total_fsi = temp[1].text.strip()

    except:
        total_fsi="NA"




    # ---------*****************part 3
    temp = part3[0].find_all('div',class_="col-md-3 col-sm-3")

    # bank name
    try:
        Bank_name = temp[1].text.strip()
    except:
        Bank_name="NA"

    # *******************
    temp = part3[1].find_all('div',class_="col-md-3 col-sm-3")

    # IFSC Code
    try:
        ifsc = temp[1].text.strip()
    except:
        ifsc="NA"

#     @@@@@
    # Address
    table_present="No"
    add = "";
    if(fsoup.find('div',id = "fldFirm")!= None):

        # Organization - Name, Contact,
        details = fsoup.find('div',id = "fldFirm").find_all('div',class_ = 'row')

        # *************** Organization
        temp = details[0].find_all('div',class_="col-md-3 col-sm-3")

        # organization
        try:
            organization = temp[1].text.strip()
        except:
            organization="NA"
        # if len(df_rera_unique[(df_rera_unique['Project Name'].str.lower().str.strip() == name.lower().strip()) & (df_rera_unique['Organization/Individual'].str.lower().str.strip() == organization.lower().strip())]) > 0:
        #     return 'Present'

        # ******************** Organization Contact Details - Contact Number
        temp = details[-2].find_all('div',class_="col-md-3 col-sm-3")

        # contact
        try:
            contact = temp[1].text.strip()
            if(contact==""):
                contact= "NA"
        except:
            contact="NA"

        # ************************** Organization Contact Details - Website URl
        temp = details[-1].find_all('div',class_="col-md-3 col-sm-3")

        # Website url
        try:
            website  = temp[1].text.strip()
            if(website == ""):
                website = "NA"
        except:
            website="NA"


        # Developer Details
        # Address Details
        developer_address = ""

        temp1 = details[-8].find_all('div',class_="col-md-3 col-sm-3")
        try:
            # Block Number
            developer_address+=temp1[1].text.strip()
            developer_address+=","

            # Building Name
            developer_address+=temp1[3].text.strip()
            developer_address+=","

            # **********************
            temp1 = details[-7].find_all('div',class_="col-md-3 col-sm-3")

            # Street Name
            developer_address+=temp1[1].text.strip()
            developer_address+=","

            # Locality
            developer_address+=temp1[3].text.strip()
            developer_address+=","

            # **********************
            temp1 = details[-6].find_all('div',class_="col-md-3 col-sm-3")

            # Land mark
            developer_address+=temp1[1].text.strip()
            developer_address+=","

            # State/UT
            developer_address+=temp1[3].text.strip()
            developer_address+=","


            #
            temp1 = details[-5].find_all('div',class_="col-md-3 col-sm-3")

            # Division
            developer_address+=temp1[1].text.strip()
            developer_address+=","

            # District
            developer_address+=temp1[3].text.strip()
            developer_address+=","

            # *********************************
            temp1 = details[-4].find_all('div',class_="col-md-3 col-sm-3")

            # Taluka
            developer_address+=temp1[1].text.strip()
            developer_address+=","

            # Village
            developer_address+=temp1[3].text.strip()
            developer_address+=","

            # ********************
            temp1 = details[-3].find_all('div',class_="col-md-3 col-sm-3")

            # Pin Code
            developer_address+=temp1[1].text.strip()
        except:
            developer_address="NA"


    # if table is another table element
    else:

        details = fsoup.find('div',id = "fldind").find_all('div',class_ = 'row')
        temp = details[0].find_all('div',class_="col-md-3 col-sm-3")

        # organization name
        try:
            organization = temp[1].text.strip()+"(Individual)"
        except:
            organization = "NA"
        # if len(df_rera_unique[(df_rera_unique['Project Name'].str.lower().str.strip() == name.lower().strip()) & (df_rera_unique['Organization/Individual'].str.lower().str.strip() == organization.lower().strip())]) > 0:
        #     return 'Present'

        # ******************** Organization Contact Details - Contact Number
        temp = details[-3].find_all('div',class_="col-md-3 col-sm-3")
        try:
            contact = temp[1].text.strip()
            if(contact==""):
                contact= "NA"
        except:
            contact = "NA"

        # ******************** Organization Contact Details - Website url
        temp = details[-1].find_all('div',class_="col-md-3 col-sm-3")
        try:
            website  = temp[1].text.strip()
            if(website == ""):
                website = "NA"
        except:
            website="NA"
        developer_address = " "

        try:
            temp1 = details[-9].find_all('div',class_="col-md-3 col-sm-3")
            developer_address+=temp1[1].text.strip()
            developer_address+=","
            developer_address+=temp1[3].text.strip()
            temp1 = details[-8].find_all('div',class_="col-md-3 col-sm-3")
            developer_address+=","
            developer_address+=temp1[1].text.strip()
            developer_address+=","
            developer_address+=temp1[3].text.strip()
            temp1 = details[-7].find_all('div',class_="col-md-3 col-sm-3")
            developer_address+=","
            developer_address+=temp1[1].text.strip()
            developer_address+=","
            developer_address+=temp1[3].text.strip()
            temp1 = details[-6].find_all('div',class_="col-md-3 col-sm-3")
            developer_address+=","
            developer_address+=temp1[1].text.strip()
            developer_address+=","
            developer_address+=temp1[3].text.strip()
            temp1 = details[-5].find_all('div',class_="col-md-3 col-sm-3")
            developer_address+=","
            developer_address+=temp1[1].text.strip()
            developer_address+=","
            developer_address+=temp1[3].text.strip()
            temp1 = details[-4].find_all('div',class_="col-md-3 col-sm-3")
            developer_address+=","
            developer_address+=temp1[1].text.strip()
        except Exception as e:
            developer_address = "NA"
#             print("developer_address", developer_address)
#     if len(df_rera_unique[(df_rera_unique['Project Name'].str.lower().str.strip() == name.lower().strip()) & (df_rera_unique['Organization/Individual'].str.lower().str.strip() == organization.lower().strip())]) > 0:
#         return 'Present'

#     print(developer_address)
    # Member Information table
    past_properties_data = []
    past_name = ""
    past_project = ""
    past_type = ""
    past_area = ""
    past_address = ""
    others = ""
    cts_number = ""
    no_past_ap = ""
    no_past_plot = ""
    past_date = ""
    past_actual = ""
    oname = ""


    # Member Information table
    oname =""
    try:
        tabl = fsoup.find('div', id = 'DivMember' ).find('table',class_ = "table")
        try:
            members = tabl.find_all('tr')

            # name of all members
            for j in range(1,len(members)):
                oname = oname+","+members[j].find('td').text.strip()
        except:
            oname="NA"
    except:
        oname="NA"

    # Past Experience Details
    Past_experience_details = {}

    # Past Experience table
    try:
        table = fsoup.find('table', class_='table table-striped grid-table')
        try:

                    # rows in the table
            rows = table.find_all('tr', class_="grid-row")
            table_present = "Yes"
            # all rows in a Table
            for row in rows:
                typ = row.find_all('td', class_="grid-cell")

                # getting Column content for ec=very row
                for t in typ:
                    a = t.get('data-name')
                    # project Name
                    if a == "ProjectName":
                        past_name = t.text.strip()

                    # Type of Project
                    elif a == "TypeofProject":
                        past_type = t.text
                        if "Residential" in past_type:
                            past_type="Residential"
                        elif "Others" in past_type:
                            past_type = "Others"
                        else:
                            past_type = "NA"

                    # others
                    elif a == "PTypeOther":
                        others = t.text.strip()
                        if(others == ""):
                            others = "NA"

                    # Land Area
                    elif a == "LandArea":
                        past_area = t.text.strip()
                        if(past_area == ""):
                            past_area = "NA"
                        else:
                            try:
                                past_area-re.sub(r'\.{2,}', '.', past_area.split("+")[0])
                                past_area = float(past_area)
                            except:
                                past_area = past_area

                    # past project address
                    elif a == "Address":
                        past_address = t.text.strip()
                        if(past_address == ""):
                            past_address = "NA"

                    # CTS No
                    elif a == "CTSNo":
                        cts_number = t.text.strip()
                        if(cts_number == ""):
                            cts_number = "NA"
#                         print("CTS No.", cts_number)

                    # Number of Buildings/Plot
                    elif a == "BuildPlotCount":
                        no_past_plot = t.text.strip()
                        if(no_past_plot == ""):
                            no_past_plot = "NA"

                        else:
                            no_past_plot = int(no_past_plot)

                    # Number of Apartments
                    elif a == "Apartmentcount":
                        no_past_ap = t.text.strip()
                        if(no_past_ap == ""):
                            no_past_ap = "NA"
                        else:
                            no_past_ap = int(no_past_ap)
#                         print("Number of Apartments",no_past_ap)

                    # Original Proposed Date of Completion
                    elif a == "ProjectStartDate":
                        past_date = t.text.strip()
                        if(past_date == ""):
                            past_date = "NA"

                    # Actual Date of Completion
                    elif a == "ProjectEndDate":
                        past_actual = t.text.strip()
                        if(past_actual == ""):
                            past_actual = "NA"
#                         print("Actual Date of Completion",past_actual)

                    past_project = {
                        "Organization": organization,
                        "Owners/Patrners" : oname,
                        "Contact":contact,
                        "Project": name,
                        "Project Name": past_name,
                        "Type of Project": past_type,
                        "Others": others,
                        "Land Area (in sqmtrs)": past_area,
                        "Address": past_address,
                        "CTS number": cts_number,
                        "Number of Buildings/Plot": no_past_plot,
                        "Number Apartments": no_past_ap,
                        "Original Proposed Date of Completion": past_date,
                        "Actual Date of Completion": past_actual,
                        "website of Organization":website
                    }
                    details = {'Contact':contact,
                                'Type of Project': past_type,
                                'Others': others,
                                'Land Area(Sq mtrs)': past_area,
                                'Address': past_address,
                                'CTS-Number': cts_number,
                                'Number of Buildings/plot': no_past_plot,
                                'Number of Apartmennts': no_past_ap,
                                'Original Proposed Date of Completion': past_date,
                                'Actual Date of Completion': past_actual,
                                'website of organization': website}
                    Past_experience_details[past_name] = details
                    df_past = df_past.append(past_project,ignore_index=True)


        except:
#             print("error in past expreience1")
            None


    except:
#         print("error in past expreience2")
        None

    df = pd.DataFrame(columns = ["Project Name","Project Status","Organization/Individual","developer_address","Proposed Date of Completion","Revised Proposed Date of Completion",
        "Litigations related to the project ?","Project type","Plot no.","Building_name","Number of Sanctioned Floors",
        "Number of Basement's","Number of Plinth","Number of Podium's","Number of Stilts","Total_no_of_open_Parking","Number_of_Closed_Parking","boundaries east",
        "boundaries west","boundaries north","boundaries south","state","division","district","taluka","village","street","locality",
        "pin code","Total Building Count","Sanctioned Buildings Count","Proposed But Not Sanctioned Buildings Count","Aggregate area(In sqmts) of recreational open space",
        "Built-up-Area as per Proposed FSI (In sqmts) ( Proposed but not sanctioned)","Built-up-Area as per Approved FSI (In sqmts)",
        "TotalFSI","bhks","Bank_name","IFSC CODE","website of Organization","Contact","Past Experience","Past_experience_details"])


    # Building Details
    tabless = fsoup.find_all('table', class_="table table-bordered table-responsive table-striped")
    try:
        build_name_list = []  # Initialize an empty list to store the output
        for table in tabless:
            rows = table.find_all('tr')  # Find all rows in the table
            for row in rows:
                columns = row.find_all('td')
                if len(columns) == 11:
                    # Extracting data from the 3rd and 4th columns in the row
                    key = columns[2].text.strip()
                    value = columns[3].text.strip()

                    # Replace empty strings with 'Not Available'
                    value = 'Not Available' if not value else value

                    # Append key-value pair as a list to the main list
                    build_name_list.append([key, value])

    except Exception as e:
        print(f"Error: {e}")

    # Print or use the final list of lists
    print(build_name_list)


    # Building Details
    # Lists to hold all building info for a single project
    headers = [h.text.strip() for h in fsoup.find_all("h2")]
    building_names = []
    NOBs = []
    NOPs = []
    NOPOs = []
    NOSs = []
    total_parking_lists = []
    total_closed_parking_lists = []
    total_floors_list = []
    all_bhks = []
    plot_details = "NA"
    BHK_wise_CA = []
    Carpet_wise_total_sold = []

    # Building Details
    if "Building Details" in headers:
        building_details = fsoup.find('div', id='DivBuilding')\
            .find('table', class_="table table-bordered table-responsive table-striped")\
            .find_all('tr')

        for j in range(0, len(building_details)):
            bhks = {}
            cbhks = {}

            try:
                if building_details[0 + j].find_all('th')[0].text == 'Sr.No.':
                    pass
                else:
                    continue
            except:
                continue

            try:
                one = building_details[0 + j].find_all('th')
                two = building_details[1 + j].find_all('td')
                three = building_details[2 + j].find_all('td')[2].find_all('tr')

                # Building Name
                try:
                    if (one[2].text.strip() == "Name (Also mention identification of building/wing/other as per approved plan)"):
                        building_names.append(two[2].text.strip())
                    else:
                        building_names.append("NA")
                except:
                    building_names.append("NA")

                # Number of Basement's
                try:
                    if (one[4].text.strip() == "Number of Basement's"):
                        NOBs.append(two[4].text.strip())
                    else:
                        NOBs.append("NA")
                except:
                    NOBs.append("NA")

                # Number of Plinth
                try:
                    if (one[5].text.strip() == "Number of Plinth"):
                        NOPs.append(two[5].text.strip())
                    else:
                        NOPs.append("NA")
                except:
                    NOPs.append("NA")

                # Number of Podium's
                try:
                    if (one[6].text.strip() == "Number of Podium's"):
                        NOPOs.append(two[6].text.strip())
                    else:
                        NOPOs.append("NA")
                except:
                    NOPOs.append("NA")

                # Number of Sanctioned Floors
                try:
                    if (one[7].text.strip() == "Number of Sanctioned Floors"):
                        total_floors_list.append(int(two[7].text.strip()))
                    else:
                        total_floors_list.append(0)
                except:
                    total_floors_list.append(0)

                # Number of Stilts
                try:
                    if (one[8].text.strip() == "Number of Stilts"):
                        NOSs.append(two[8].text.strip())
                    else:
                        NOSs.append("NA")
                except:
                    NOSs.append("NA")

                # Total open parking
                try:
                    if (one[9].text.strip()):
                        total_parking_lists.append(two[9].text.strip())
                    else:
                        total_parking_lists.append("NA")
                except:
                    total_parking_lists.append("NA")

                # Total closed parking
                try:
                    if (one[10].text.strip()):
                        total_closed_parking_lists.append(two[10].text.strip())
                    else:
                        total_closed_parking_lists.append("NA")
                except:
                    total_closed_parking_lists.append("NA")

                # BHK details
                bhks = {}
                cbhks = {}
                for row in range(1, len(three)):
                    try:
                        inner = three[row].find_all('td')
                        bhk_type = inner[1].text.strip()

                        if bhk_type in bhks:
                            bhks[bhk_type]['total'] += int(inner[3].text)
                            bhks[bhk_type]['sold'] += int(inner[4].text)
                            bhks[bhk_type]['avg_area'] = (
                                (bhks[bhk_type]['avg_area'] * cbhks[bhk_type]) +
                                float(re.sub(r'\.{2,}', '.', inner[2].text.split("+")[0]))
                            ) / (cbhks[bhk_type] + 1)
                            cbhks[bhk_type] += 1
                        else:
                            cbhks[bhk_type] = 1
                            bhks[bhk_type] = {
                                'total': int(inner[3].text),
                                'sold': int(inner[4].text),
                                'avg_area': float(re.sub(r'\.{2,}', '.', inner[2].text.split("+")[0]))
                            }
                    except Exception as e:
                        print("BHK Error:", e)
                        continue
                all_bhks.append(bhks)

            except Exception as e:
                print("Outer loop error:", e)
                continue
        #BHK wise CA
    #     try:
        try:
            all_tables = fsoup.find_all('table', class_="table table-bordered table-responsive table-striped")
            bhk_tables = []

            # Filter only BHK-wise tables based on presence of Apartment Type & Carpet Area headers
            for table in all_tables:
                headers = [th.text.strip().lower() for th in table.find_all('th')]
                if 'apartment type' in headers and 'carpet area (in sqmts)' in headers:
                    bhk_tables.append(table)

        except Exception as e:
            print(f"[ERROR] While filtering BHK tables: {e}")

        # --- BHK-wise Carpet Area ---
        try:
            
            for table in bhk_tables:
                wing_dict = {}
                rows = table.find_all('tr')[1:]  # Skip header
                for row in rows:
                    cols = row.find_all('td')
                    if len(cols) == 5:
                        apt_type = cols[1].text.strip().lower()
                        carpet_area = cols[2].text.strip()
                        wing_dict.setdefault(apt_type, []).append(carpet_area)
                if wing_dict:
                    BHK_wise_CA.append(wing_dict)

        except Exception as e:
            print(f"[ERROR] While extracting BHK-wise carpet area: {e}")

        # --- Carpet-wise Total vs Sold Apartments ---
        try:
            for table in bhk_tables:
                wing_dict = {}
                rows = table.find_all('tr')[1:]  # Skip header
                for row in rows:
                    cols = row.find_all('td')
                    if len(cols) == 5:
                        apt_type = cols[1].text.strip().lower()
                        carpet_area = cols[2].text.strip()
                        total_apts = int(cols[3].text.strip())
                        booked_apts = int(cols[4].text.strip())

                        if apt_type not in wing_dict:
                            wing_dict[apt_type] = {}

                        if carpet_area not in wing_dict[apt_type]:
                            wing_dict[apt_type][carpet_area] = [0, 0]

                        wing_dict[apt_type][carpet_area][0] += total_apts
                        wing_dict[apt_type][carpet_area][1] += booked_apts

                if wing_dict:
                    Carpet_wise_total_sold.append(wing_dict)

        except Exception as e:
            print(f"[ERROR] While extracting carpet-wise total/booked data: {e}")
        
    if "Plot Details" in headers:
        try:
            all_tables = fsoup.find_all("table", class_="table table-bordered table-responsive table-striped")
            plot_table = None
            for table in all_tables:
                headers_row = table.find('tr')
                if headers_row and any("Plot" in th.text for th in headers_row.find_all('th')):
                    plot_table = table
                    break
            if plot_table:
                plot_details = [[cell.text.strip() for cell in row.find_all('td')] for row in plot_table.find_all('tr')[1:]]
            else:
                plot_details = "NA"
        except (AttributeError, IndexError) as e:
            print(f"Error parsing Plot Details: {e}")
            plot_details = "NA"



    project={
    # "rera_id":ids[i],
        "Project Name":name,
        "Project Status":status,
        # "latitude":lat[i],
        # "longitude":lng[i],
        "Organization/Individual":organization,
        "developer_address":developer_address,
        "Proposed Date of Completion":date,
        "Revised Proposed Date of Completion":rdate,
        "Litigations related to the project ?":litigations,
        "Project type": type,
        "Plot no.":no,
        "Building_name":building_names,
        "Number of Sanctioned Floors":total_floors_list,
        "Number of Basement's":NOBs,
        "Number of Plinth":NOPs,
        "Number of Podium's":NOPOs,
        "Number of Stilts":NOSs,
        "Total_no_of_open_Parking":total_parking_lists,
        "Number_of_Closed_Parking":total_closed_parking_lists,
        "boundaries east":east,
        "boundaries west":west,
        "boundaries north":north,
        "boundaries south":south,
        "state":state,
        "division":division,
        "district":district,
        "taluka":taluka,
        "village":village,
        "street":street,
        "locality":locality,
        "pin code":pincode,
        "Total Plot/Project area (sqmts)":area,
        "Total Building Count":total_building,
        "Sanctioned Buildings Count":sanc_count,
        "Proposed But Not Sanctioned Buildings Count":notsanc_count,
        "Aggregate area(In sqmts) of recreational open space":agg_area,
        "Sactioned FSI of the Project Applied for Registration (Sanctioned Build-up Area)":FSI_area,
        "Built-up-Area as per Proposed FSI (in sqfts)(Proposed but not Sanctioned)":FSI_area_approved,
        "Permissible Built-up Area":total_fsi,
        "bhks":all_bhks,
        "Bank_name":Bank_name,
        "IFSC CODE":ifsc,
        "website of Organization":website,
        "Contact":contact,
        "Past Experience":table_present,
        "Past_experience_details":Past_experience_details,
        "plot_details_in_No_of_Plot/Area_of_plot/Booked_plot": plot_details,
        "Building_wise_completion_date":build_name_list,
        "BHK_wise_CA":BHK_wise_CA,
        "Carpet_wise_total_sold":Carpet_wise_total_sold
    }
    # df=pd.DataFrame(project)
    if isinstance(project, dict):
        project = pd.DataFrame([project])
    
    # Use pd.concat() to append the new row(s)
    df = pd.concat([df, project], ignore_index=True)

#         print(building_data_list)

    return df_past,df


In [ ]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

option = webdriver.ChromeOptions()
driver = webdriver.Chrome(options = option)

def process_image(image_path):
    """Enhance the image for better OCR accuracy and return the OCR result."""
    image = Image.open(image_path)
    image = image.convert('L')  # Convert to grayscale
    image = image.filter(ImageFilter.MedianFilter())  # Apply median filter
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(2)  # Increase contrast

    # Save the enhanced image
    enhanced_image_path = 'captcha_enhanced.png'
    image.save(enhanced_image_path)

    # Use OCR to read captcha
    captcha_text = pytesseract.image_to_string(image).strip()
    return captcha_text

def clean_captcha(captcha_text):
    """Remove unwanted characters from the OCR result."""
    return re.sub(r'[^a-zA-Z0-9]', '', captcha_text)

try:
    # Load the website
    link = "https://maharerait.mahaonline.gov.in/searchlist/search?MenuID=1069"
    driver.get(link)
    driver.maximize_window()
    time.sleep(2)
    # Click on registration dropdown
    a = driver.find_element(By.ID, 'Promoter')
    a.click()
    # # Click on Register real estate agent
    # b = driver.find_element(By.XPATH, "/html/body/header/div[5]/div/div/div/ul/li[5]/ul/li[1]/a")
    # b.click()
    # # Handle pop-up alert
    # driver.switch_to.alert.accept()
    # driver.switch_to.window(driver.window_handles[1])
    # time.sleep(5)
    # Click on advance search
    advance_search = driver.find_element(By.XPATH, "/html/body/section/div/div/form/div[2]/div[2]/div[8]/div/div/div[1]/input")
    advance_search.click()
    time.sleep(2)

    # Select user type
    bubble = driver.find_element(By.XPATH, "/html/body/section/div/div/form/div[2]/div[2]/div[1]/div/div[2]/input[1]")
    bubble.click()

    # Select state
    state = driver.find_element(By.XPATH,"/html/body/section/div/div/form/div[2]/div[2]/div[4]/div/div[2]/select/option[4]")
    state.click()
    time.sleep(2)
    # Select district
    district = driver.find_element(By.XPATH,"/html/body/section/div/div/form/div[2]/div[2]/div[5]/div[1]/div[2]/select/option[27]")
    district.click()

    while True:
        # Wait for the captcha image to load
        captcha_image_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'CaptchaImage'))
        )

        # Screenshot the captcha image
        captcha_image_path = 'captcha_screenshot.png'
        captcha_image_element.screenshot(captcha_image_path)

        # Attempt to read the CAPTCHA using OCR
        captcha_text = process_image(captcha_image_path)
        cleaned_captcha_text = clean_captcha(captcha_text)
        print(f"Raw OCR Captcha text: {captcha_text}")
        print(f"Cleaned Captcha text: {cleaned_captcha_text}")

        # Enter captcha
        try:
            captcha_input = driver.find_element(By.ID, 'CaptchaInputText')
            captcha_input.clear()
            captcha_input.send_keys(cleaned_captcha_text)
            print(cleaned_captcha_text)
        except (ElementNotInteractableException, NoSuchElementException) as e:
            print(f"Error entering CAPTCHA text: {e}")
            continue

        # Click on search
        try:
            submit = driver.find_element(By.ID, "btnSearch")
            submit.click()
        except (ElementNotInteractableException, NoSuchElementException) as e:
            print(f"Error clicking search button: {e}")
            continue

        time.sleep(5)

        try:
            # Check if the "Captcha is not valid" popup appears
            error_popup = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'sweet-alert'))
            )
            if error_popup.is_displayed():
                print("Invalid CAPTCHA, retrying...")
                ok_button = error_popup.find_element(By.CLASS_NAME, 'confirm')
                ok_button.click()
            else:
                break
        except TimeoutException:
            # If no error popup, break the loop
            break

    # Continue with your further code...


finally:
    print("Captcha accepted")
    pass

# settings to change in driver opened window
1. open the autodownload option
2. change the download directory path

In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium.webdriver.common.by import By
import pandas as pd
import os
from time import sleep

# Directory to store downloads
download_directory = r"D:\rera code"
os.makedirs(download_directory, exist_ok=True)

rera_final = pd.DataFrame()

# Start time
start_time = time.time()

def wait_for_download(filepath, timeout=30):
    """Waits for the file to be completely downloaded."""
    start = time.time()
    while time.time() - start < timeout:
        if os.path.exists(filepath) and os.path.getsize(filepath) > 0:
            return True
        sleep(1)
    return False

def process_project(result, i):
    """Function to scrape and process one project."""
    global rera_final  # To update the DataFrame across threads
    
    try:
        # Extract RERA ID
        name = result.find_elements(By.CSS_SELECTOR, '[title="Download Certificate"]')[0].get_attribute('data-docname') + ".pdf"
        rera_id = name[:-4]
        
        # Skip existing RERA IDs
        if not df_rera_unique[df_rera_unique['RERA ID'].str.lower().str.strip().eq(rera_id.lower().strip())].empty:
            return f"Skipping RERA ID {rera_id}, already exists."

        # Extract Last Modified Date
        last_modified_date = result.find_element(By.CSS_SELECTOR, 'td[data-name="lastModifiedDate"]').text
        Promoter_name = result.find_element(By.CSS_SELECTOR, 'td[data-name="Name"]').text


        # Download the certificate
        result.find_elements(By.CSS_SELECTOR, '[title="Download Certificate"]')[0].click()
        filepath = os.path.join(download_directory, name)
        
        if not wait_for_download(filepath):
            return f"Download timed out for RERA ID {rera_id}. Skipping PDF processing."

        # Extract dates from PDF
        date = ["NA", "NA", "NA"]
        try:
            date = extract_dates_from_pdf(filepath)
            os.remove(filepath)
        except Exception as e:
            print(f"Error extracting date from PDF for {rera_id}: {e}")

        # Extract project details
        try:
            href = result.find_elements(By.CLASS_NAME, "grid-cell")[4].find_element(By.TAG_NAME, "a").get_attribute("href")
            fsoup = project_detail_page(href)
            df_past, df_final = rera_data(fsoup)
        except Exception as e:
            return f"Error extracting project details for {rera_id}: {e}"

        # Add additional columns
        df_final['Rera_certificate_bottom_left_date'] = date[0]
        df_final['Rera_commmencing_from'] = date[1]
        df_final['Rera_commmencing_ending_with'] = date[2]
        df_final['Promoter Name'] = Promoter_name
        df_final['Last_Modified_Date'] = last_modified_date
        df_final['RERA ID'] = rera_id

        # Append to the final DataFrame using concat
        rera_final = pd.concat([rera_final, df_final], ignore_index=True)

        # Store data in CSV
        past_csv_path = os.path.join(download_directory, "o1a_20241231_Past_final_review_Pune.csv")
        final_csv_path = os.path.join(download_directory, "o1b_20241231_rera_final_review_Pune.csv")
        
        df_past.to_csv(past_csv_path, mode='a', index=False, header=not os.path.isfile(past_csv_path))
        df_final.to_csv(final_csv_path, mode='a', index=False, header=not os.path.isfile(final_csv_path))

        return f"Successfully processed RERA ID {rera_id}"
    
    except Exception as e:
        return f"Error processing project {i + 1}: {e}"


# Web Scraping Execution
page_count = 1
project_count = 0  # Counter for processed projects

while True:
    try:
        result = driver.find_element(By.XPATH, "/html/body/section/div/div/form/div[3]/div/div[2]/div[1]/div/table/tbody")
        results = result.find_elements(By.CLASS_NAME, "grid-row")
    except Exception as e:
        print("Error locating table rows:", e)
        break

    if page_count > 0:
        with ThreadPoolExecutor(max_workers=10) as executor:  # Limit max workers to 10 for better efficiency
            future_to_project = {executor.submit(process_project, results[i], i): i for i in range(len(results))}
            
            for future in as_completed(future_to_project):
                print(future.result())  # Print status of each project
                project_count += 1  # Increment count of processed projects
                
                if project_count >= 50:  # Stop after processing 50 projects
                    break

        print(f"Page Number {page_count} DONE")

    if project_count >= 50:
        break  # Stop processing after 50 projects

    # Move to the next page
    page_count += 1
    go_to_next_page = input("Enter any key to proceed to the next page (or 'q' to quit): ").strip().lower()
    
    if go_to_next_page == 'q':
        break
    
    sleep(5)

# End time
end_time = time.time()

# Total time taken
elapsed_time = end_time - start_time
print(f"\nTotal Time Taken to Scrape 50 Projects: {elapsed_time:.2f} seconds")


In [ ]:
import pandas as pd
df=pd.read_csv(r"D:\Rera_Scraping_March_25\o1b_20241231_rera_final_review_Thane.csv")
df

--- 

### 🔧 **Final Execution: Parity Check Between BHK and Carpet Data (Before & After Fixes)**

This cell:
- Extracts total, sold, and count values from `bhks` and original `Carpet_wise_total_sold`
- Saves the **original carpet and CA data** before processing
- **Drops the 0th index** from both `Carpet_wise_total_sold` and `BHK_wise_CA` if list length > 1 (to remove duplicates)
- Aligns `{}` entries in `Carpet_wise_total_sold` and `BHK_wise_CA` based on blank entries in `bhks`
- Recalculates all carpet stats **after corrections**
- Adds:
  - `parity_before` → compares bhk with carpet before fix
  - `parity_after` → compares bhk with carpet after fix
- Outputs cleaned columns for manual cross-verification

--- 

In [ ]:
import pandas as pd
import ast

# Load the DataFrame
df = pd.read_excel(r"D:\Rera_Scraping_March_25\Demo_bhk_wise_CA.xlsx")

# Step 1: Parse JSON safely
def safe_literal_eval(x):
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return [] if isinstance(x, str) and x.strip().startswith("[") else {}

cols_to_parse = [
    "BHKs_Updated",
    "Carpet_wise_total_sold_converted_avg_converted",
    "BHK_wise_CA_Updated"
]

for col in cols_to_parse:
    df[col] = df[col].apply(safe_literal_eval)

# Step 2: Extract initial totals
def extract_all_counts(row):
    bhk_total = bhk_sold = 0
    bhk_data = row["BHKs_Updated"]
    if isinstance(bhk_data, list):
        for b in bhk_data:
            if isinstance(b, dict):
                for unit in b.values():
                    bhk_total += unit.get("total", 0)
                    bhk_sold += unit.get("sold", 0)

    carpet_total = carpet_sold = 0
    carpet_data = row["Carpet_wise_total_sold_converted_avg_converted"]
    if isinstance(carpet_data, list):
        for entry in carpet_data:
            if isinstance(entry, dict):
                for flat in entry.values():
                    for count in flat.values():
                        carpet_total += count[0]
                        carpet_sold += count[1]

    return pd.Series([
        bhk_total, bhk_sold, len(bhk_data) if isinstance(bhk_data, list) else 0,
        carpet_total, carpet_sold, len(carpet_data) if isinstance(carpet_data, list) else 0
    ])

df[[
    "bhk_total", "bhk_sold", "bhk_count",
    "carpet_total_before", "carpet_sold_before", "carpet_count_before"
]] = df.apply(extract_all_counts, axis=1)

# Step 3: Drop 0th index from carpet and CA (if needed)
df["Carpet_wise_total_sold_converted_avg_converted"] = df["Carpet_wise_total_sold_converted_avg_converted"].apply(
    lambda x: x[1:] if isinstance(x, list) and len(x) > 1 else x
)
df["BHK_wise_CA_Updated"] = df["BHK_wise_CA_Updated"].apply(
    lambda x: x[1:] if isinstance(x, list) and len(x) > 1 else x
)

# Step 4: Fix alignment if bhk_count > carpet_count
def fix_alignment(row):
    bhk_len = len(row["BHKs_Updated"]) if isinstance(row["BHKs_Updated"], list) else 0
    carpet = row["Carpet_wise_total_sold_converted_avg_converted"]
    ca = row["BHK_wise_CA_Updated"]

    if isinstance(carpet, list) and isinstance(ca, list):
        while len(carpet) < bhk_len:
            carpet.append({})
        while len(ca) < bhk_len:
            ca.append({})

    row["Carpet_wise_total_sold_converted_avg_converted"] = carpet
    row["BHK_wise_CA_Updated"] = ca
    return row

df = df.apply(fix_alignment, axis=1)

# Step 5: Recompute carpet totals after alignment
def extract_carpet_after(row):
    total = sold = 0
    data = row["Carpet_wise_total_sold_converted_avg_converted"]
    if isinstance(data, list):
        for entry in data:
            if isinstance(entry, dict):
                for flat in entry.values():
                    for count in flat.values():
                        total += count[0]
                        sold += count[1]
    return pd.Series([total, sold, len(data)])

df[["carpet_total_after", "carpet_sold_after", "carpet_count_after"]] = df.apply(extract_carpet_after, axis=1)

# Step 6: Parity checks
df["parity_before"] = (
    (df["bhk_total"] == df["carpet_total_before"]) &
    (df["bhk_sold"] == df["carpet_sold_before"]) &
    (df["bhk_count"] == df["carpet_count_before"])
)

df["parity_after"] = (
    (df["bhk_total"] == df["carpet_total_after"]) &
    (df["bhk_sold"] == df["carpet_sold_after"]) &
    (df["bhk_count"] == df["carpet_count_after"])
)


In [ ]:
df.to_excel(r"D:\Rera_Scraping_March_25\Demo_bhk_wise_CA_parity.xlsx",index = False)

In [ ]:
df['parity_after'].unique()

In [ ]:
df.to_csv(r'D:\RERA op\rera main code\o1d_berfore_manual.csv',index=False)

In [ ]:
import pandas as pd
df=pd.read_excel(r"D:\RERA op\rera main code\demoo.xlsx",sheet_name='Sheet2')
df

#MANUAL PROCESS

# SQFT_to_SQMT Conversion

In [ ]:
import pandas as pd
import ast

#File and sheet info
# file_path = r"D:\Rera_Scraping_March_25\Thane\o1b_20241231_rera_final_review_Thane_before_Processing.xlsx"

#Load CSV
# df = pd.read_excel(file_path)

# # Clean column names
# df.columns = df.columns.str.strip()

# Target columns
# column_name = "Carpet_wise_total_sold"
# unit_column = "cwts(sqm/sqft)"

# Helper to check if value is numeric
def is_numeric_string(value):
    try:
        float(value)
        return True
    except:
        return False

# Main conversion logic with merge logic for duplicate sqm keys
def convert_sqft_to_sqmt(data_str):
    try:
        data = ast.literal_eval(data_str)
    except:
        return data_str  # Return as-is if not parseable

    # Check for "3 bhk"
    has_3bhk = "3bhk" in data_str.lower()
    min_sqft = 150 if has_3bhk else 100

    def convert_keys(obj):
        if isinstance(obj, dict):
            new_dict = {}
            for k, v in obj.items():
                if isinstance(v, dict):
                    new_inner = {}
                    for inner_k, inner_v in v.items():
                        if is_numeric_string(inner_k) and float(inner_k) >= min_sqft:
                            sqm = round(float(inner_k) * 0.092903, 2)
                            sqm_key = str(sqm)
                            if sqm_key in new_inner:
                                new_inner[sqm_key] = [
                                    new_inner[sqm_key][0] + inner_v[0],
                                    new_inner[sqm_key][1] + inner_v[1],
                                ]
                            else:
                                new_inner[sqm_key] = inner_v
                        else:
                            if inner_k in new_inner:
                                new_inner[inner_k] = [
                                    new_inner[inner_k][0] + inner_v[0],
                                    new_inner[inner_k][1] + inner_v[1],
                                ]
                            else:
                                new_inner[inner_k] = inner_v
                    new_dict[k] = new_inner
                else:
                    new_dict[k] = v
            return new_dict
        return obj

    return str([convert_keys(d) for d in data])

# Apply only where 'sqft' is mentioned
def conditional_convert(row):
    unit = str(row.get(unit_column, "")).strip().lower()
    data = row.get(column_name, "")
    if unit == "sqft":
        return convert_sqft_to_sqmt(data)
    else:
        return data  # untouched

# Apply conversion
df[column_name + "_converted"] = df.apply(conditional_convert, axis=1)

#Save result to file
output_path = r"D:\Rera_Scraping_March_25\Thane\o1b_20241231_rera_final_review_Thane_sqft_converted.xlsx"
df.to_excel(output_path, index=False)

print(f"✅ Done! Duplicate sqm keys merged correctly.\nConverted where 'sqft' specified.\nSaved to:\n{output_path}")
 

# Sum_Problem_Threshold

In [ ]:
import pandas as pd
import ast

# # File path and sheet
# file_path = r"D:\Rera_Scraping_March_25\Thane\o1b_20241231_rera_final_review_Thane_sqft_converted.xlsx"

# # Load Excel
# df = pd.read_excel(file_path)
# df.columns = df.columns.str.strip()

# # Column names
# column_name = "Carpet_wise_total_sold_converted"
# unit_column = "cwts(sum/avg)"

# # Function to divide area by count (only for 'sum' rows)
def divide_area_by_count(data_str, is_3bhk=False):
    try:
        data = ast.literal_eval(data_str)
    except:
        return data_str  # return original if parsing fails

    min_val = 105 if is_3bhk else 63

    def transform_keys(obj):
        if isinstance(obj, dict):
            new_dict = {}
            for k, v in obj.items():
                if isinstance(v, dict):
                    new_inner = {}
                    for inner_k, inner_v in v.items():
                        try:
                            key_float = float(inner_k)
                            if isinstance(inner_v, list) and inner_v[0] != 0 and key_float > min_val:
                                divided = round(key_float / inner_v[0], 2)
                                new_inner[str(divided)] = inner_v
                            else:
                                new_inner[inner_k] = inner_v
                        except:
                            new_inner[inner_k] = inner_v
                    new_dict[k] = new_inner
                else:
                    new_dict[k] = v
            return new_dict
        return obj

    return str([transform_keys(d) for d in data])

# Apply only if 'cwts(sum/avg)' == 'sum'
def conditional_divide(row):
    unit = str(row.get(unit_column, "")).strip().lower()
    data = row.get(column_name, "")
    is_3bhk = "3bhk" in str(row).lower()
    if unit == "sum":
        return divide_area_by_count(data, is_3bhk)
    else:
        return data  # keep original if not 'sum'

# Create new column with result
df[column_name + "_avg_converted"] = df.apply(conditional_divide, axis=1)

#Save to file
output_path = r"D:\Rera_Scraping_March_25\Thane\o1b_20241231_rera_final_review_Thane_Sum_converted_demo.xlsx"
df.to_excel(output_path, index=False)

print(f"✅ Done! Values divided by count for 'sum' rows with min thresholds.\nSaved to:\n{output_path}")


In [ ]:
df

In [ ]:
import pandas as pd

df=pd.read_excel(r"D:\Rera_Scraping_March_25\Thane\o1b_20241231_rera_final_review_Thane_Sum_converted_demo.xlsx")
# df

#do manual for mix and apartment number in CWTS & create "Carpet_wise_total_sold_converted_manually"

In [ ]:
df["Carpet_wise_total_sold_converted_avg_converted"] = df["Carpet_wise_total_sold_converted_avg_converted"].str.replace("..", ".", regex=False)

In [ ]:
import pandas as pd
import ast
import numpy as np

# Load the file
# df = pd.read_excel(r"D:\Rera_Scraping_March_25\Thane\o1b_20241231_rera_final_review_Thane_Sum_converted_demo.xlsx")

# Function to safely parse stringified data
def safe_literal_eval(x):
    if pd.isna(x):  # Handle NaN or None
        return {}  # Return an empty dict or adjust based on your needs
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return {}  # Return an empty dict for invalid strings or adjust as needed

# Parse stringified lists/dicts
df["Carpet_wise_total_sold_converted_avg_converted"] = df["Carpet_wise_total_sold_converted_avg_converted"].apply(safe_literal_eval)

# Fix BHK_wise_CA structure based on Carpet_wise_total_sold_converted_manually
def fix_bhk_wise_ca_format(row):
    fixed = []
    # Ensure row is a list or dict, handle empty/invalid cases
    if not isinstance(row, (list, dict)):
        return fixed
    for carpet_dict in row:
        new_entry = {}
        for bhk_type, ca_dict in carpet_dict.items():
            if isinstance(ca_dict, dict):
                new_entry[bhk_type] = [str(k) for k in ca_dict.keys()]
        fixed.append(new_entry)
    return fixed

# Apply the function to create the updated column
df["BHK_wise_CA_Updated"] = df["Carpet_wise_total_sold_converted_avg_converted"].apply(fix_bhk_wise_ca_format)

# Display the dataframe

In [ ]:
df.to_excel(r"D:\Rera_Scraping_March_25\Thane\demo_1.xlsx", index = False)

In [ ]:
import pandas as pd
import ast

df = pd.read_excel(r"D:\Rera_Scraping_March_25\Thane\demo_1.xlsx")

# Function to create Desire_output
def generate_desire_output(carpet_data):
    # Convert string to Python object if it's a string
    if isinstance(carpet_data, str):
        try:
            carpet_data = ast.literal_eval(carpet_data)
        except (ValueError, SyntaxError):
            return []  # Return empty list if parsing fails

    output = []
    for item in carpet_data:
        bhk_result = {}
        for bhk, carpet_dict in item.items():
            total_units = 0
            sold_units = 0
            area_keys = []

            for area_str, counts in carpet_dict.items():
                try:
                    area = float(area_str.strip())
                except:
                    continue
                total_units += counts[0]
                sold_units += counts[1]
                area_keys.append(area)

            avg_area = sum(area_keys) / len(area_keys) if area_keys else 0
            bhk_result[bhk.upper()] = {
                "total": total_units,
                "sold": sold_units,
                "avg_area": avg_area
            }

        output.append(bhk_result)
    return output

# Apply transformation
df["bhks_Updated"] = df["Carpet_wise_total_sold_converted_avg_converted"].apply(generate_desire_output)

In [ ]:
df.to_excel(r"D:\Rera_Scraping_March_25\Thane\demo_2.xlsx", index = False)

In [ ]:
# df = pd.read_excel(r"D:\Rera_Scraping_March_25\Demo_bhk_wise_CA_&_BHKs.xlsx")


# Calculate parityA
def extract_totals(row):
    # Parse bhks_Updated if it's a string
    bhk_data = row["bhks_Updated"]
    if isinstance(bhk_data, str):
        try:
            bhk_data = ast.literal_eval(bhk_data)
        except (ValueError, SyntaxError):
            bhk_data = []  # Return empty list if parsing fails

    bhk_total = bhk_sold = 0
    for b in bhk_data:
        for unit in b.values():
            bhk_total += unit.get("total", 0)
            bhk_sold += unit.get("sold", 0)

    # Parse Carpet_wise_total_sold_converted_avg_converted if it's a string
    carpet_data = row["Carpet_wise_total_sold_converted_avg_converted"]
    if isinstance(carpet_data, str):
        try:
            carpet_data = ast.literal_eval(carpet_data)
        except (ValueError, SyntaxError):
            carpet_data = []  # Return empty list if parsing fails

    carpet_total = carpet_sold = 0
    for b in carpet_data:
        for flat in b.values():
            for count in flat.values():
                carpet_total += count[0]
                carpet_sold += count[1]

    bhk_count = len(bhk_data)
    carpet_count = len(carpet_data)
    return pd.Series([bhk_total, bhk_sold, bhk_count, carpet_total, carpet_sold, carpet_count])

df[["bhk_total_after_manual", "bhk_sold_after_manual", "bhk_count_after_manual", 
    "carpet_total_after_manual", "carpet_sold_after_manual", "carpet_count_after_manual"]] = df.apply(extract_totals, axis=1)

In [ ]:
# If bhks and Building_name are stored as strings, convert them
df["bhks_Updated"] = df["bhks_Updated"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df["Building_name"] = df["Building_name"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Now process the row to compute [[building_name, total, sold], ...]
def get_building_units_list_format(row):
    bhks = row["bhks_Updated"]
    names = row["Building_name"]
    result = []
    for i in range(len(names)):
        total = sold = 0
        if i < len(bhks) and isinstance(bhks[i], dict):
            for unit_data in bhks[i].values():
                total += unit_data.get("total", 0)
                sold += unit_data.get("sold", 0)
        result.append([names[i], total, sold])
    return result

# Apply the function
df["Building_wise_total_sold_units_after_manual"] = df.apply(get_building_units_list_format, axis=1)
# Compute total and sold units across all buildings in each list
def compute_totals_from_list(building_list):
    total_units = sum(item[1] for item in building_list)
    sold_units = sum(item[2] for item in building_list)
    return pd.Series([total_units, sold_units])

df[["Total_units_sum_after_manual", "Sold_units_sum_after_manual"]] = df["Building_wise_total_sold_units_after_manual"].apply(compute_totals_from_list)
#df

In [ ]:
df.to_excel(r"D:\Rera_Scraping_March_25\Thane\demo_3.xlsx", index = False)

In [ ]:
def check_parity(row):
    return (
        row["bhk_total_after_manual"] == row["Total_units_sum_after_manual"] and
        row["carpet_total_after_manual"] == row["Total_units_sum_after_manual"] and
        row["bhk_sold_after_manual"] == row["Sold_units_sum_after_manual"] and
        row["carpet_sold_after_manual"] == row["Sold_units_sum_after_manual"]
    )

# Add parity column
df["final_parity_after_manual"] = df.apply(check_parity, axis=1)
df

In [ ]:
df['final_parity_after_manual'].unique()

In [ ]:
df.to_excel(r"D:\Rera_Scraping_March_25\Thane\o1b_20241231_rera_final_review_Thane_Manually_Done.xlsx", index= False)

In [ ]:
print(df[df['final_parity_after_manual']==False]['Carpet_wise_total_sold_converted_refined'])

In [ ]:
def check_parity(row):
    return (
        row["Total_units_sum_after_manual"] == row["carpet_total_after"] and
        row["Sold_units_sum_after_manual"] == row["carpet_sold_after"] 
    )

# Add parity column
df["final_parity_between_old_and_new"] = df.apply(check_parity, axis=1)
df

In [ ]:
df.to_excel('01c_after_.xlsx',index=False)

# Find out total supply and consume area

In [ ]:
import ast
import re
import pandas as pd

def clean_carpet_size(value):
    match = re.search(r'\d+\.\d+|\d+', value)
    return float(match.group()) if match else 0.0

def calculate_areas(cell):
    total_supply = 0.0
    total_consumed = 0.0
    try:
        parsed = ast.literal_eval(cell)
        for unit in parsed:
            for bhk_type, carpet_data in unit.items():
                for carpet_size, counts in carpet_data.items():
                    size_float = clean_carpet_size(carpet_size)
                    supply_count = counts[0]
                    consumed_count = counts[1]
                    total_supply += size_float * supply_count
                    total_consumed += size_float * consumed_count
    except Exception as e:
        print(f"Error parsing cell: {e}")
    return total_supply, total_consumed


# Apply function
df[['total_supply_area', 'total_consumed_area']] = df['Carpet_wise_total_sold_converted_avg_converted'].apply(
    lambda x: pd.Series(calculate_areas(x))
)

In [ ]:
import pandas as pd

df=pd.read_excel(r"D:\Rera_Scraping_March_25\Thane\o1b_20241231_rera_final_review_Thane_Manually_Done.xlsx")
df

In [ ]:
rera_df = df[['Project Name', 'Project Status', 'Organization/Individual',
       'developer_address', 'Proposed Date of Completion',
       'Revised Proposed Date of Completion',
       'Litigations related to the project ?', 'Project type', 'Plot no.',
       'Building_name', 'Number of Sanctioned Floors', "Number of Basement's",
       'Number of Plinth', "Number of Podium's", 'Number of Stilts',
       'Total_no_of_open_Parking', 'Number_of_Closed_Parking',
       'boundaries east', 'boundaries west', 'boundaries north',
       'boundaries south', 'state', 'division', 'district', 'taluka',
       'village', 'street', 'locality', 'pin code', 'Total Building Count',
       'Sanctioned Buildings Count',
       'Proposed But Not Sanctioned Buildings Count',
       'Aggregate area(In sqmts) of recreational open space',
       'Sactioned FSI of the Project Applied for Registration (Sanctioned Build-up Area)',
       'Built-up-Area as per Proposed FSI (in sqfts)(Proposed but not Sanctioned)', 'Permissible Built-up Area',
       'Bank_name', 'IFSC CODE', 'website of Organization', 'Contact',
       'Past Experience', 'Past_experience_details',
       'Total Plot/Project area (sqmts)',
       'plot_details_in_No_of_Plot/Area_of_plot/Booked_plot',
       'Building_Name', 'Rera_certificate_bottom_left_date',
       'Rera_commmencing_from', 'Rera_commmencing_ending_with',
       'Promoter Name', 'Last_Modified_Date', 'RERA ID',
       'Building_wise_total_sold_units_after_manual',
       'Carpet_wise_total_sold_converted_avg_converted',
       'BHK_wise_CA_Updated',  'bhks_Updated','Total_units_sum_after_manual', 'Sold_units_sum_after_manual']]
rera_df

In [ ]:
rera_df.rename(columns={
    'Carpet_wise_total_sold_converted_refined':'Carpet_wise_total_sold',
    'BHK_wise_CA_fixed':'BHK_wise_CA',
    'bhks_Desire_output':'bhks',
    'Total_units_sum_after_manual':'total_units',
    'Sold_units_sum_after_manual':'total_sold_units',
    'Building_wise_total_sold_units_after_manual':'Building_wise_total_sold'
},inplace=True)
rera_df

In [ ]:
rera_df['Project Name']=rera_df['Project Name'].str.strip().str.title()
rera_df['Organization/Individual']=rera_df['Organization/Individual'].str.strip().str.title()

In [ ]:
import ast

def safe_flatten(series):
    result = []
    for item in series:
        if isinstance(item, list):
            result.extend(item)
        elif isinstance(item, str):
            try:
                parsed = ast.literal_eval(item)
                if isinstance(parsed, list):
                    result.extend(parsed)
                else:
                    result.append(parsed)
            except:
                result.append(item)
        else:
            result.append(item)
    return result

aggregation_functions = {
    'Project Name':'first', 'Project Status': "first", 'Organization/Individual':'first',
    'developer_address':'first', 'Proposed Date of Completion': "first",
    'Revised Proposed Date of Completion': "first", 'Litigations related to the project ?': "first",
    'Project type': "first", 'Plot no.':"first",

    'Building_name': safe_flatten,
    'Number of Sanctioned Floors': safe_flatten,
    "Number of Basement's": safe_flatten,
    'Number of Plinth': safe_flatten,
    "Number of Podium's": safe_flatten,
    'Number of Stilts': safe_flatten,
    'Total_no_of_open_Parking': safe_flatten,
    'Number_of_Closed_Parking': safe_flatten,

    'boundaries east':'first', 'boundaries west':'first', 'boundaries north':'first',
    'boundaries south':'first', 'state':'first', 'division':'first', 'district':'first', 'taluka':'first',
    'village':'first', 'street':'first', 'locality':'first', 'pin code':'first',

    'Total Building Count':'sum',
    'Sanctioned Buildings Count':'sum',
    'Proposed But Not Sanctioned Buildings Count':'sum',
    'Aggregate area(In sqmts) of recreational open space':'sum',
    'Sactioned FSI of the Project Applied for Registration (Sanctioned Build-up Area)':'sum',
    'Built-up-Area as per Proposed FSI (in sqfts)(Proposed but not Sanctioned)':'sum', 'Permissible Built-up Area':'sum',

    'Bank_name':'first', 'IFSC CODE':'first', 'website of Organization':'first', 'Contact':'first',
    'Past Experience':'first', 'Past_experience_details':'first',
    'Total Plot/Project area (sqmts)':'sum',

    'plot_details_in_No_of_Plot/Area_of_plot/Booked_plot': safe_flatten,
    'Building_Name': safe_flatten,
    'Building_wise_total_sold': safe_flatten,
    'Carpet_wise_total_sold_converted_avg_converted': safe_flatten,
    'BHK_wise_CA_Updated': safe_flatten,
    'bhks_Updated': safe_flatten,
    'RERA ID': safe_flatten,

    'Rera_certificate_bottom_left_date':'first',
    'Rera_commmencing_from':'first',
    'Rera_commmencing_ending_with':'first',
    'Promoter Name':'first',
    'Last_Modified_Date':'first',
    'total_units':'sum',
    'total_sold_units':'sum'
}

df_rera_unique = (
    rera_df.groupby(['Project Name', 'Organization/Individual'])
    .aggregate(aggregation_functions)
    .reset_index(drop=True)
)
df_rera_unique

In [ ]:
df_rera_unique['Building_name']

In [ ]:
print(df_rera_unique['total_units'].sum())
print(rera_df['total_units'].sum())
print(df_rera_unique['total_sold_units'].sum())
print(rera_df['total_sold_units'].sum())


In [ ]:
df_rera_unique['rera_entry'] = df_rera_unique['RERA ID'].apply(lambda x: len(x) if isinstance(x, list) else 0)
df_rera_unique

In [ ]:
df_rera_unique['rera_entry'].sum()

In [ ]:
df_rera_unique.to_excel(r'D:\Rera_Scraping_March_25\Thane\o1g_final_agreegated.xlsx',index=False)

In [ ]:
df_grand_excel=pd.read_excel(r"D:\RERA op\rera main code\Pune RERA GRAND EXCEL VERSION 9 Temp.xlsx",sheet_name='Sheet1')
df_grand_excel

In [ ]:
import ast

# Optional: create the columns if not already present
df_rera_unique['Location updated'] = None
df_rera_unique['Project_lat'] = None
df_rera_unique['Project_lng'] = None

# Iterate over grand excel rows
for _, row in df_grand_excel.iterrows():
    # Split multiline RERA Project field
    projects = [p.strip() for p in str(row['RERA Project']).split("\n") if p.strip()]
    
    # Parse Organization list
    try:
        org_list = ast.literal_eval(row['Organization/Individual'])
    except:
        org_list = [row['Organization/Individual']]
    
    # Location and lat/lng values
    location = row.get('Modified RERA Location - v3')
    lat = row.get('Project_lat')
    lng = row.get('Project_lng')

    # Matching condition
    condition = (df_rera_unique['Project Name'].isin(projects)) &                 (df_rera_unique['Organization/Individual'].isin(org_list))
    
    # Apply updates
    df_rera_unique.loc[condition, 'Location updated'] = location
    df_rera_unique.loc[condition, 'Project_lat'] = lat
    df_rera_unique.loc[condition, 'Project_lng'] = lng
df_rera_unique

In [ ]:
# Find duplicate combinations in df_grand_excel
dupes = df_grand_excel.groupby(['RERA Project', 'Organization/Individual']).size().reset_index(name='count')
dupes = dupes[dupes['count'] > 1]

dupes

In [ ]:
df_rera_unique

In [ ]:
df_rera_unique.to_excel('pune_final_rera_excel.xlsx',index=False)

In [ ]:
import pandas as pd
df=pd.read_excel(r"D:\RERA op\rera main code\o1g_20250426_Pune_rera_with_location_manual.xlsx")
df

In [ ]:
df.columns = (
    df.columns
      .str.replace('\xa0', ' ', regex=False)  # turn NBSP into normal space
      .str.strip()                             # remove leading/trailing spaces
)


In [ ]:
df['Project Name']=df['Project Name'].str.strip().str.title()
df['Location updated']=df['Location updated'].str.strip().str.title()

In [ ]:
df['Location updated']

In [ ]:
import ast

def safe_flatten(series):
    result = []
    for item in series:
        if isinstance(item, list):
            result.extend(item)
        elif isinstance(item, str):
            try:
                parsed = ast.literal_eval(item)
                if isinstance(parsed, list):
                    result.extend(parsed)
                else:
                    result.append(parsed)
            except:
                result.append(item)
        else:
            result.append(item)
    return result

aggregation_functions = {
    'Project Name':'first', 'Project Status': "first", 'Organization/Individual':'first',
    'developer_address':'first', 'Proposed Date of Completion': "first",
    'revised date of completion': "first", 'Litigations related to the project ?': "first",
    'Project type': "first", 'Plot no.':"first",

    'Building_name': safe_flatten,
    'Number of Sanctioned Floors': safe_flatten,
    "Number of Basement's": safe_flatten,
    'Number of Plinth': safe_flatten,
    "Number of Podium's": safe_flatten,
    'Number of Stilts': safe_flatten,
    'Total_no_of_open_Parking': safe_flatten,
    'Number_of_Closed_Parking': safe_flatten,

    'boundaries east':'first', 'boundaries west':'first', 'boundaries north':'first',
    'boundaries south':'first', 'state':'first', 'division':'first', 'district':'first', 'taluka':'first',
    'village':'first', 'street':'first', 'locality':'first', 'pin code':'first',

    'Total Building Count':'sum',
    'Sanctioned Buildings Count':'sum',
    'Proposed But Not Sanctioned Buildings Count':'sum',
    'Aggregate area(In sqmts) of recreational open space':'sum',
    'Built-up-Area as per Proposed FSI (In sqmts) ( Proposed but not sanctioned)':'sum',
    'Built-up-Area as per Approved FSI (In sqmts)':'sum', 'TotalFSI':'sum',

    'Bank_name':'first', 'IFSC CODE':'first', 'website of Organization':'first', 'Contact':'first',
    'Past Experience':'first', 'Past_experience_details':'first',
    'Total Plot/Project area (sqmts)':'sum',

    'plot_details_in_No_of_Plot/Area_of_plot/Booked_plot': safe_flatten,
    'Building_wise_completion_date': safe_flatten,
    'Building_wise_total_sold': safe_flatten,
    'Carpet_wise_total_sold': safe_flatten,
    'BHK_wise_CA': safe_flatten,
    'bhks': safe_flatten,
    'RERA ID': safe_flatten,

    'Commencement bottom left date':'first',
    'Commencement Date':'first',
    'Rera_commmencing_ending_with':'first',
    'Promoter Name':'first',
    'Last_Modified_Date':'first',
    'total_units':'sum',
    'total_sold_units':'sum',
    'rera_entry':'sum',
    'Project_lat':'first',
    'Project_lng':'first',
    'Location updated':'first'
    
}

df_rera_unique = (
    df.groupby(['Project Name', 'Location updated'])
    .aggregate(aggregation_functions)
    .reset_index(drop=True)
)
df_rera_unique

In [ ]:
df_rera_unique.to_excel(r"D:\RERA op\rera main code\o1h_20250426_Pune_rera_with_location_manual.xlsx")

### Add RERA ID, LAT, and LNG

In [ ]:
url = "https://maharerait.mahaonline.gov.in/GIS/getData.ashx?GetMapData=Data"
print(url)
# go to this url and copy all data
# save copied data into a json file

In [ ]:
import requests

url = 'https://maharerait.mahaonline.gov.in/GIS/getData.ashx?GetMapData=Data'

response = requests.get(url, verify=False)
data = response.json()

# Save data to a file
filename = r'o1d_20240811_THANE_rera_by-direct-link.json'  # Replace with your desired file name
with open(filename, 'w') as f:
    json.dump(data,f)

In [ ]:
# convert json to xlsx
jsn = pd.read_json(filename)
jsn.to_csv(r"D:\RERA op\rera main code\o1e_20240811_THANE_rera_by-direct-link.csv",index=False)

In [ ]:
df_direct_with_link = pd.read_csv(r"D:\RERA op\rera main code\o1e_20240811_THANE_rera_by-direct-link.csv",index_col=[0])
df_direct_with_link

In [ ]:
# filter columns
df_filtered = df_direct_with_link.loc[:, df_direct_with_link.columns.intersection(['Name_of_Project', 'Project_District', 'Application_No', 'lat','lng'])]
print("Shape",df_filtered.shape)
df_filtered.head()

In [ ]:
# filter data by district/City
df_pune_coords = df_filtered[(df_filtered['Project_District'] == 'Pune')].reset_index(drop=True)
df_pune_coords

In [ ]:
# make mumbai lat, lng data unique
df_mumbai_coords_unique = (df_pune_coords.groupby(['Name_of_Project', 'Project_District', 'Application_No', 'lat', 'lng']).aggregate({'Name_of_Project':'first', 'Project_District':'first', 'Application_No':'first', 'lat':'first', 'lng':'first'})).reset_index(drop=True)

In [ ]:
df_mumbai_coords_unique['Name_of_Project']=df_pune_coords['Name_of_Project'].apply(lambda x:x.title())
df['Project Name']=df['Project Name'].apply(lambda x:x.title())
df['locality']=df['locality'].apply(lambda x:str(x).title())
df.shape

In [ ]:
# adding latitude, longitude columns in df_rera_unique from df_mumbai_coords_unique
df['rera_id'] = ""
df['lat'] = ""
df['lng'] = ""
for i in range(len(df)):
    pn1 = df['Project Name'][i]
    loc1= df['locality'][i]
    result1 = df_mumbai_coords_unique[df_mumbai_coords_unique['Name_of_Project'] == pn1]
    if result1.shape[0] != 0:
        rera_id = result1.iloc[0]['Application_No']
        lat = result1.iloc[0]['lat']
        lng = result1.iloc[0]['lng']

    else:
        rera_id = "na"
        lat = "na"
        lng = "na"

    df['rera_id'][i] = rera_id
    df['lat'][i] = lat
    df['lng'][i] = lng

In [ ]:
df

In [ ]:
df.to_excel(r"D:\RERA op\rera main code\o1f_20240811_THANE_rera_with_rera_coords.xlsx")

In [ ]:
df.info()

In [ ]:
df